In [ ]:
import os
# os.add_dll_directory("C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.0/bin")

import tensorflow as tf
import numpy as np
import PIL
import matplotlib.pyplot as plt
import pathlib
from tensorflow.keras.layers.experimental import preprocessing

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

batch_size = 32
img_height = 180
img_width = 180
seed=256
AUTOTUNE = tf.data.AUTOTUNE


print("Creating training ds...")

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=data_dir,
    batch_size=batch_size,
    image_size=(img_height, img_width),
    validation_split=0.2,
    subset="training",
    seed=seed
)

class_names = train_ds.class_names
print(class_names)

train_ds = train_ds.cache().shuffle(100).prefetch(AUTOTUNE)

print("train ds creation successful, creating valid_ds...")

valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=data_dir,
    batch_size=batch_size,
    image_size=(img_height, img_width),
    validation_split=0.2,
    subset="validation",
    seed=seed
)

valid_ds = valid_ds.cache().prefetch(AUTOTUNE)

print("valid ds created successfully, plotting sample images...")


plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

print("plotting sample images successful, creating model...")

preprocessing_section = tf.keras.models.Sequential([
    preprocessing.Rescaling(1/255., input_shape=(img_width, img_height, 3)),
    preprocessing.RandomZoom(0.1),
    preprocessing.RandomRotation(0.1),
    preprocessing.RandomFlip("horizontal",
                             input_shape=(img_height,
                                          img_width,
                                          3))
])

model = tf.keras.models.Sequential([
    preprocessing_section,
    tf.keras.layers.Conv2D(32, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5)
])

print("model creation successful, now compiling it...")

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics="accuracy"
)

print("model compilation successful, now fitting model...")

epochs = 20
history = model.fit(train_ds, epochs=epochs, validation_data=valid_ds)

print("model is successfully fit, now plotting results...")

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

print("Finished plotting results onto graph, program is successful")





